In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statistics as es 
import csv
import re 
import warnings
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## ENEM  DATASET

In [2]:
#Choose just useful features for us 
fenem= ['NU_INSCRICAO','CO_ESCOLA','Q001','Q002','Q003','Q004','Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011','Q012','Q013','Q014','Q015','Q016','Q017','Q019','Q022','Q024','Q027', 'TP_COR_RACA',
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO',
       'IN_TREINEIRO', 'TP_ST_CONCLUSAO', 'TP_ESCOLA', 'TP_ENSINO']
enem17a=pd.read_csv("~/data/Enem/MICRODADOS_ENEM_2017.csv", sep=';',encoding="iso-8859-2", usecols = fenem)
enem18a=pd.read_csv("~/data/Enem/MICRODADOS_ENEM_2018.csv", sep=';',encoding="iso-8859-2", usecols = fenem)

In [3]:
category=(['TP_COR_RACA'])
enem17a[category]= enem17a[category].astype(str)
enem18a[category]= enem17a[category].astype(str)

In [4]:
enem17 = enem17a.copy()
enem18 = enem18a.copy()


In [5]:
#How many Students left? How many schools are they in?
print ("There are", enem17.shape[0], "students",  "of", enem17.CO_ESCOLA.value_counts().count() ,"Schools in ENEM 2017")
#enem17.head()

print ("There are", enem18.shape[0], "students",  "of", enem18.CO_ESCOLA.value_counts().count() ,"Schools in ENEM 2018")
#enem18.head()

There are 6731341 students of 33834 Schools in ENEM 2017
There are 5513747 students of 32253 Schools in ENEM 2018


In [6]:
#enem18.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])

### FILTER DOMAIN 

In [7]:
#ps: May have intersections
print("Will be dropped" , (enem18.CO_ESCOLA.isnull()).sum(), 'who does have school enrollment')
print("Will be dropped" , (enem18.IN_TREINEIRO == 1).sum(), 'who are just trainning')
print("Will be dropped" , (enem18.TP_ST_CONCLUSAO!= 2).sum(), 'who aren´t on last secoNdary school year')
print("Will be dropped" , (enem18.TP_ESCOLA!=2).sum(), 'who does not study in public school')
print("Will be dropped" , (enem18.TP_ENSINO !=1).sum(), 'who does not have regular curriculum')

Will be dropped 4064926 who does have school enrollment
Will be dropped 527709 who are just trainning
Will be dropped 3873648 who aren´t on last secoNdary school year
Will be dropped 4376256 who does not study in public school
Will be dropped 2384237 who does not have regular curriculum


In [8]:
#Drop out those whithout schools ,
enem17.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])
enem18.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])
#more filters
enem17 = enem17[(enem17.IN_TREINEIRO !=1) & (enem17.TP_ST_CONCLUSAO==2) & (enem17.TP_ESCOLA==2) & (enem17.TP_ENSINO ==1)]
enem18 = enem18[(enem18.IN_TREINEIRO !=1) & (enem18.TP_ST_CONCLUSAO==2) & (enem18.TP_ESCOLA==2) & (enem18.TP_ENSINO ==1)]

In [9]:
enem18.TP_ENSINO.describe()

count    1009156.0
mean           1.0
std            0.0
min            1.0
25%            1.0
50%            1.0
75%            1.0
max            1.0
Name: TP_ENSINO, dtype: float64

In [10]:
#Second, drop out insufficient scores 
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
# With NaN
enem17.dropna(inplace = True, axis=0, subset=fields)
enem18.dropna(inplace = True, axis=0, subset=fields)
# With 0
enem17 = enem17[~(enem17[fields] == 0).any(axis=1)]
enem18 = enem18[~(enem18[fields] == 0).any(axis=1)]

In [11]:
#Only schools that have more than ten students in ENEM 1
enem17['STUDENT_COUNT'] = enem17.groupby(['CO_ESCOLA'])['CO_ESCOLA'].transform('count') 
enem17 = enem17[(enem17['STUDENT_COUNT']>= 10)]
enem17.drop(['STUDENT_COUNT'], axis=1,inplace=True)

enem18['STUDENT_COUNT'] = enem18.groupby(['CO_ESCOLA'])['CO_ESCOLA'].transform('count') 
enem18 = enem18[(enem18['STUDENT_COUNT']>= 10)]
enem18.drop(['STUDENT_COUNT'], axis=1,inplace=True)

In [12]:
print(enem17.shape)
print(enem18.shape)

(934645, 31)
(733248, 31)


#### Set last quartile of average scores as the best students for the 2017 dataset

In [13]:
#Creating average of students in all fields
enem17['AVG_SCORE'] = (enem17.NU_NOTA_CN + enem17.NU_NOTA_CH + enem17.NU_NOTA_LC + enem17.NU_NOTA_MT + enem17.NU_NOTA_REDACAO)/5
enem18['AVG_SCORE'] = (enem18.NU_NOTA_CN + enem18.NU_NOTA_CH + enem18.NU_NOTA_LC + enem18.NU_NOTA_MT + enem18.NU_NOTA_REDACAO)/5

In [14]:
#Creating the average of all schools by each studentes score
enem17['AVG_SCORE_SCHOOL'] = enem17.groupby(['CO_ESCOLA'])['AVG_SCORE'].transform('mean') 
enem18['AVG_SCORE_SCHOOL'] = enem18.groupby(['CO_ESCOLA'])['AVG_SCORE'].transform('mean') 

In [15]:
#Transform in School grain before to separate the scores in quartiles
temp17 = enem17.drop_duplicates(['CO_ESCOLA'])
temp17['TARGET'] = pd.qcut (temp17.AVG_SCORE_SCHOOL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 

temp18 = enem18.drop_duplicates(['CO_ESCOLA'])
temp18['TARGET'] = pd.qcut (temp18.AVG_SCORE_SCHOOL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 

In [16]:
#It's right? 
print((temp17.TARGET==0).sum()/(temp17.TARGET.count())*100, '% lowers quartis in ENEM 2017')
print((temp18.TARGET==0).sum()/(temp18.TARGET.count())*100, '% lowers quartis in ENEM 2018')

74.99845018907693 % lowers quartis in ENEM 2017
75.00170056458745 % lowers quartis in ENEM 2018


In [17]:
print(temp17.shape)
print(temp18.shape)

(16131, 34)
(14701, 34)


In [18]:
#turn back in Student grain
enem17= pd.merge(temp17[['CO_ESCOLA', 'TARGET']], enem17, left_on='CO_ESCOLA', right_on='CO_ESCOLA', how='right')
enem18= pd.merge(temp18[['CO_ESCOLA', 'TARGET']], enem18, left_on='CO_ESCOLA', right_on='CO_ESCOLA', how='right')

In [19]:
#check if the same length as before
print(enem17.shape)
print(enem18.shape)

(934645, 34)
(733248, 34)


In [20]:
enem17.head(2)

,CO_ESCOLA,TARGET,NU_INSCRICAO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q019,Q022,Q024,AVG_SCORE,AVG_SCORE_SCHOOL
0,35001259.0,0,170001665543,3,2,2,1.0,0,452.8,451.3,529.1,439.1,520.0,E,E,B,B,F,A,B,C,A,A,B,B,B,A,B,A,C,D,B,478.46,493.553043
1,35001259.0,0,170003346964,1,2,2,1.0,0,491.9,418.9,513.4,553.3,500.0,E,E,E,B,H,A,B,B,A,A,B,A,B,A,B,A,B,C,B,495.50,493.553043


In [21]:
enem18.head(2)

,CO_ESCOLA,TARGET,NU_INSCRICAO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q019,Q022,Q024,AVG_SCORE,AVG_SCORE_SCHOOL
0,23077140.0,0,180008257068,1,2,2,1.0,0,411.8,429.8,445.2,459.0,340.0,D,D,C,B,C,A,B,C,A,A,B,A,B,A,A,A,B,B,A,417.16,486.252444
1,23077140.0,0,180008309669,3,2,2,1.0,0,457.3,564.2,474.6,462.9,300.0,D,D,C,B,C,A,B,C,A,A,B,A,B,A,A,A,B,B,A,451.80,486.252444


## TEACHERS CENSUS DATASET 

In [22]:
#Choose just useful features for us now, improve memory use
fcenso = ['CO_PESSOA_FISICA','NU_ANO_CENSO','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']

#Reading data of Teachers in Censo 2017
CO17=pd.read_csv("~/data/censo2017/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
NE17=pd.read_csv("~/data/censo2017/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
NT17=pd.read_csv("~/data/censo2017/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
SE17=pd.read_csv("~/data/censo2017/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
SU17=pd.read_csv("~/data/censo2017/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)

#There are differences in feature names
fcenso18 = ['ID_DOCENTE','NU_ANO_CENSO','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']

CO18=pd.read_csv("~/data/censo2018/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
NE18=pd.read_csv("~/data/censo2018/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
NT18=pd.read_csv("~/data/censo2018/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
SE18=pd.read_csv("~/data/censo2018/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
SU18=pd.read_csv("~/data/censo2018/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)


In [23]:
#Concatening the subsets and creating a unique dataset, than transform all features and categories, as it are.
censo17a= pd.concat([CO17, NE17, NT17, SE17, SU17])
censo17 = censo17a.copy()

censo18a= pd.concat([CO18, NE18, NT18, SE18, SU18])
censo18 = censo18a.copy()
#censo17=censo17.astype('category')

Get only teachers who work in secondary schools (based on ENEM)

In [24]:
#Some informations
print ("There are", censo17.drop_duplicates('CO_PESSOA_FISICA').shape[0], "Teachers",  "at", censo17.CO_ENTIDADE.value_counts().count() ,"Schools in Census 2017")

print ("There are", censo18.drop_duplicates('ID_DOCENTE').shape[0], "Teachers",  "at", censo18.CO_ENTIDADE.value_counts().count() ,"Schools in Census 2018")

There are 2603186 Teachers at 185925 Schools in Census 2017
There are 2581297 Teachers at 183746 Schools in Census 2018


In [25]:
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
censo17.drop_duplicates(subset=['CO_PESSOA_FISICA', 'CO_ENTIDADE'], inplace=True)
censo18.drop_duplicates(subset=['ID_DOCENTE', 'CO_ENTIDADE'], inplace=True)

In [26]:
#getting the schools represented in ENEM
schoolsEnem17 = enem17.drop_duplicates('CO_ESCOLA')
schoolsEnem17 = schoolsEnem17[(['CO_ESCOLA','TARGET'])]
#schoolsEnem17 = schoolsEnem17.astype(str)

schoolsEnem18 = enem18.drop_duplicates('CO_ESCOLA')
schoolsEnem18 = schoolsEnem18[(['CO_ESCOLA','TARGET'])]
#schoolsEnem18 = schoolsEnem18.astype(str)


In [51]:
schoolsEnem18.count()

CO_ESCOLA    14701
TARGET       14701
dtype: int64

In [27]:
schoolsEnem18.CO_ESCOLA.isin(censo18.CO_ENTIDADE).sum()

14687

In [28]:
#Merge the two datasets, whereas that only teachers who work in school represented in ENEM 17 and 18
censo17= pd.merge(schoolsEnem17, censo17, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

censo18= pd.merge(schoolsEnem18, censo18, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

In [29]:
print(censo17.shape)
print(censo18.shape)

(619249, 8)
(559941, 8)


In [30]:
censo18.drop_duplicates('CO_ENTIDADE').count()

CO_ESCOLA            14687
TARGET               14687
NU_ANO_CENSO         14687
ID_DOCENTE           14687
IN_ESPECIALIZACAO    13865
IN_MESTRADO          13865
IN_DOUTORADO         13865
CO_ENTIDADE          14687
dtype: int64

In [31]:
print(censo17.isnull().sum())
print(censo18.isnull().sum())

CO_ESCOLA                0
TARGET                   0
NU_ANO_CENSO             0
CO_PESSOA_FISICA         0
IN_ESPECIALIZACAO    39012
IN_MESTRADO          39012
IN_DOUTORADO         39012
CO_ENTIDADE              0
dtype: int64
CO_ESCOLA                0
TARGET                   0
NU_ANO_CENSO             0
ID_DOCENTE               0
IN_ESPECIALIZACAO    28967
IN_MESTRADO          28967
IN_DOUTORADO         28967
CO_ENTIDADE              0
dtype: int64


In [32]:
#Missing values in these cases are "0" 
censo17['IN_ESPECIALIZACAO'].fillna(0.0, inplace=True)
censo17['IN_MESTRADO'].fillna(0.0, inplace=True)
censo17['IN_DOUTORADO'].fillna(0.0, inplace=True)

censo18['IN_ESPECIALIZACAO'].fillna(0.0, inplace=True)
censo18['IN_MESTRADO'].fillna(0.0, inplace=True)
censo18['IN_DOUTORADO'].fillna(0.0, inplace=True)

In [33]:
#In database have different patterns to represent level of education. ex: Master has signed in two different ways, or marked in Master and Especialization or just in Master.
def education_level(df):
    if (df.IN_ESPECIALIZACAO==0.0) & (df.IN_MESTRADO==0.0) & (df.IN_DOUTORADO ==0.0):
        #print(0)
        df['TITULACAO']= "0"
    elif (df.IN_ESPECIALIZACAO==1.0) & (df.IN_MESTRADO==0.0) & (df.IN_DOUTORADO ==0.0):
        #print(1)
        df['TITULACAO']= "1"
    elif (df.IN_ESPECIALIZACAO==1.0) & (df.IN_MESTRADO==1.0) & (df.IN_DOUTORADO ==0.0):
        #print(2)
        df['TITULACAO']= "2"
    elif (df.IN_ESPECIALIZACAO==0.0) & (df.IN_MESTRADO==1.0) & (df.IN_DOUTORADO ==0.0):
        #print(2)
        df['TITULACAO']= "2"
    else:
        #print(3)
        df['TITULACAO']= "3"  
    return df

In [34]:
censo17=censo17.apply(education_level, axis=1)

censo18=censo18.apply(education_level, axis=1)

In [35]:
censo17['TARGET'] = censo17['TARGET'].astype('int64')

In [36]:
censo17.head(2)

,CO_ESCOLA,TARGET,NU_ANO_CENSO,CO_PESSOA_FISICA,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,CO_ENTIDADE,TITULACAO
0,35001259.0,0,2017.0,1.151817e+11,1.0,0.0,0.0,35001259.0,1
1,35001259.0,0,2017.0,1.222428e+11,1.0,0.0,0.0,35001259.0,1


In [37]:
censo18.head(2)

,CO_ESCOLA,TARGET,NU_ANO_CENSO,ID_DOCENTE,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,CO_ENTIDADE,TITULACAO
0,23077140.0,0,2018,00E3AA8026744AEEC6D35E11B082B9D9,0.0,0.0,0.0,23077140,0
1,23077140.0,0,2018,0FA5A09CB34A5FEC043158EE31F812F6,0.0,0.0,0.0,23077140,0


##### We will transform the Categorical ENEM database features from student grain to school grain (Decision Grain).  Instead the pratical use of MODE, it will use the regression on the attribute distribution, having its histogram relative frequencies as input and the  decision level TARGET as output.  

In [38]:
#Creating the dataframe with the relative frequencies and target. Ready to applying regeression
def relative_frequencies(df, colum):
    root = (['CO_ESCOLA', 'TARGET'])
    df = df [root + [colum]]
    df = pd.get_dummies(df, columns=[colum])
    filter_col = [col for col in df if col.startswith(colum)]
    print (filter_col)
    for x in filter_col:            
        df[x] = df.groupby(df['CO_ESCOLA'])[x].transform(sum)/ df.groupby(df['CO_ESCOLA'])[x].transform('count')
        
    return df

In [39]:
def LR_student_to_school (df1, df2, columns):
    result = pd.DataFrame()
    for x in columns:
        temp = relative_frequencies(df1,df1[x].name)
        temp.drop_duplicates('CO_ESCOLA', inplace=True)
        temp = temp.loc[:, ~temp.columns.isin(['CO_ESCOLA'])]
        temp_DATA = temp.loc[:, temp.columns != 'TARGET']
        temp_TARGET = temp.loc[:, temp.columns == 'TARGET']

        
        LR_temp = LogisticRegression()
        LR_temp.fit(temp_DATA, temp_TARGET)
      
        
        temp = relative_frequencies(df2,df2[x].name)
        temp.drop_duplicates('CO_ESCOLA', inplace=True) 
        temp = temp.reset_index(drop=True)
        temp2 = temp.loc[:, ~temp.columns.isin(['CO_ESCOLA'])]
        temp_DATA = temp2.loc[:, temp2.columns != 'TARGET']
        
        result[x] = LR_temp.predict_proba(temp_DATA)[:,1]
        
       
    result = pd.merge(result, temp.CO_ESCOLA, left_index=True, right_index=True)
    
    return result, LR_temp

In [40]:
category_enem = (['Q001','Q002','Q003','Q004', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011',
                  'Q012','Q013','Q014','Q015','Q016','Q017','Q019','Q022','Q024','Q027', 'TP_COR_RACA'])
trans_enem_att, model_enem =LR_student_to_school(enem17, enem18, category_enem)

['Q001_A', 'Q001_B', 'Q001_C', 'Q001_D', 'Q001_E', 'Q001_F', 'Q001_G', 'Q001_H']
['Q001_A', 'Q001_B', 'Q001_C', 'Q001_D', 'Q001_E', 'Q001_F', 'Q001_G', 'Q001_H']
['Q002_A', 'Q002_B', 'Q002_C', 'Q002_D', 'Q002_E', 'Q002_F', 'Q002_G', 'Q002_H']
['Q002_A', 'Q002_B', 'Q002_C', 'Q002_D', 'Q002_E', 'Q002_F', 'Q002_G', 'Q002_H']
['Q003_A', 'Q003_B', 'Q003_C', 'Q003_D', 'Q003_E', 'Q003_F']
['Q003_A', 'Q003_B', 'Q003_C', 'Q003_D', 'Q003_E', 'Q003_F']
['Q004_A', 'Q004_B', 'Q004_C', 'Q004_D', 'Q004_E', 'Q004_F']
['Q004_A', 'Q004_B', 'Q004_C', 'Q004_D', 'Q004_E', 'Q004_F']
['Q006_A', 'Q006_B', 'Q006_C', 'Q006_D', 'Q006_E', 'Q006_F', 'Q006_G', 'Q006_H', 'Q006_I', 'Q006_J', 'Q006_K', 'Q006_L', 'Q006_M', 'Q006_N', 'Q006_O', 'Q006_P', 'Q006_Q']
['Q006_A', 'Q006_B', 'Q006_C', 'Q006_D', 'Q006_E', 'Q006_F', 'Q006_G', 'Q006_H', 'Q006_I', 'Q006_J', 'Q006_K', 'Q006_L', 'Q006_M', 'Q006_N', 'Q006_O', 'Q006_P', 'Q006_Q']
['Q007_A', 'Q007_B', 'Q007_C', 'Q007_D']
['Q007_A', 'Q007_B', 'Q007_C', 'Q007_D']
['Q008_A

In [41]:
trans_enem_att

,Q001,Q002,Q003,Q004,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q019,Q022,Q024,TP_COR_RACA,CO_ESCOLA
0,0.283931,0.157451,0.156909,0.351506,0.116585,0.206097,0.069676,0.130245,0.045315,0.267349,0.232611,0.092682,0.246421,0.184577,0.059075,0.225524,0.108885,0.145857,0.008273,0.250447,23077140.0
1,0.020776,0.033769,0.066963,0.061874,0.073213,0.206097,0.174587,0.119861,0.064260,0.254763,0.271122,0.097124,0.083448,0.189583,0.090719,0.225524,0.263952,0.171994,0.022215,0.176708,23265795.0
2,0.071532,0.061691,0.089349,0.042417,0.052493,0.218031,0.070148,0.176983,0.074495,0.247557,0.232611,0.109271,0.057278,0.190988,0.106929,0.225524,0.082639,0.067151,0.017159,0.222590,25077759.0
3,0.070324,0.108594,0.083509,0.059945,0.082846,0.217162,0.133414,0.200174,0.089842,0.249688,0.206573,0.149960,0.115857,0.222453,0.119952,0.225524,0.189647,0.149398,0.044728,0.220640,22022040.0
4,0.396745,0.388033,0.301175,0.181556,0.303349,0.280790,0.426694,0.202109,0.382995,0.237708,0.236431,0.445859,0.400365,0.274222,0.341398,0.225524,0.289761,0.438642,0.320059,0.185516,35907224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14696,0.019126,0.016940,0.062016,0.026372,0.003976,0.206097,0.049958,0.025095,0.045315,0.267349,0.134824,0.119115,0.077182,0.184577,0.057810,0.225524,0.108885,0.005116,0.010633,0.206509,15048500.0
14697,0.083311,0.043642,0.083750,0.029935,0.029083,0.206097,0.045461,0.406405,0.045315,0.267349,0.232611,0.085245,0.046390,0.184577,0.057810,0.225524,0.108885,0.005167,0.007652,0.206029,21272778.0
14698,0.000205,0.000008,0.083750,0.029935,0.001242,0.206097,0.058402,0.008607,0.045315,0.267349,0.034800,0.085245,0.046390,0.184577,0.057810,0.225524,0.026671,0.006399,0.007652,0.152864,21418209.0
14699,0.000561,0.000122,0.083750,0.029935,0.062230,0.206097,0.035280,0.100899,0.045315,0.258168,0.001477,0.085245,0.046390,0.184577,0.057810,0.225524,0.012810,0.005751,0.007652,0.199847,12023710.0


In [42]:
category_censo = (['TITULACAO'])
trans_censo_att, model_censo=LR_student_to_school(censo17, censo18, category_censo)

['TITULACAO_0', 'TITULACAO_1', 'TITULACAO_2', 'TITULACAO_3']
['TITULACAO_0', 'TITULACAO_1', 'TITULACAO_2', 'TITULACAO_3']


In [43]:
trans_censo_att

,TITULACAO,CO_ESCOLA
0,0.225611,23077140.0
1,0.189290,23265795.0
2,0.367590,25077759.0
3,0.259014,22022040.0
4,0.251320,35907224.0
...,...,...
14682,0.362122,15048500.0
14683,0.198454,21272778.0
14684,0.190689,21418209.0
14685,0.185643,12023710.0


In [44]:
#trans_enem_att.to_csv("trans_enem_att.csv")
#trans_censo_att.to_csv("trans_censo_att.csv")

In [52]:
trans_regress_att = pd.merge(trans_enem_att, trans_censo_att , left_on='CO_ESCOLA', right_on='CO_ESCOLA', how='right')

In [56]:
trans_regress_att.to_csv("trans_regress_att.csv")